In [1]:
import bcolz
import numpy as np

table = bcolz.open("D:\\OpenSourceTradePlatform\\atspy\\rqalpha\\bundle\\indexes.bcolz", 'r')

In [2]:
table

ctable((1132884,), [('date', '<u4'), ('open', '<u4'), ('close', '<u4'), ('high', '<u4'), ('low', '<u4'), ('volume', '<u8'), ('total_turnover', '<u8')])
  nbytes: 38.89 MB; cbytes: 57.42 MB; ratio: 0.68
  cparams := cparams(clevel=5, shuffle=1, cname='lz4', quantize=0)
  rootdir := 'D:\OpenSourceTradePlatform\atspy\rqalpha\bundle\indexes.bcolz'
[ (20130702, 49364843, 49214200, 49373610, 48512568, 17517169500, 1209203143)
 (20130703, 49076836, 49167840, 49265439, 48533415, 21554442300, 1426791526)
 (20130704, 48812128, 49877200, 50341233, 48248504, 21968230700, 1768369520)
 ...,
 (20170602, 39748920, 39686090, 39893720, 39566340, 1554278150, 21267079199)
 (20170605, 39595350, 39320840, 39636810, 39191070, 1843604557, 22096096415)
 (20170606, 39258800, 39686600, 39692010, 39209150, 1178144750, 17749909925)]

In [3]:
table.names

['date', 'open', 'close', 'high', 'low', 'volume', 'total_turnover']

In [4]:
table[0]

(20130702, 49364843, 49214200, 49373610, 48512568, 17517169500, 1209203143)

In [9]:
linemap = table.attrs['line_map']
linemap

{u'000001.XSHG': [1023086, 1026102],
 u'000002.XSHG': [716571, 719587],
 u'000003.XSHG': [550899, 553915],
 u'000004.XSHG': [694333, 697349],
 u'000005.XSHG': [735053, 738069],
 u'000006.XSHG': [768838, 771854],
 u'000007.XSHG': [764510, 767526],
 u'000008.XSHG': [931183, 934199],
 u'000009.XSHG': [589180, 592196],
 u'000010.XSHG': [199599, 202615],
 u'000011.XSHG': [338844, 341860],
 u'000012.XSHG': [111423, 114439],
 u'000015.XSHG': [484570, 487586],
 u'000016.XSHG': [346548, 349564],
 u'000017.XSHG': [613218, 615992],
 u'000018.XSHG': [152283, 155299],
 u'000019.XSHG': [300041, 302332],
 u'000020.XSHG': [453843, 456049],
 u'000021.XSHG': [760957, 763373],
 u'000025.XSHG': [184969, 187984],
 u'000026.XSHG': [1052671, 1055686],
 u'000027.XSHG': [435505, 438520],
 u'000028.XSHG': [155299, 158314],
 u'000029.XSHG': [448800, 451815],
 u'000030.XSHG': [779305, 782320],
 u'000031.XSHG': [406798, 409813],
 u'000032.XSHG': [985924, 988939],
 u'000033.XSHG': [208383, 211398],
 u'000034.XSHG':

In [11]:
s, e = linemap["000001.XSHG"]
s,e

(1023086, 1026102)

In [12]:
fields = table.names[1:]
fields

['open', 'close', 'high', 'low', 'volume', 'total_turnover']

In [13]:
fields[1]

'close'

In [15]:
table.cols["date"]

carray((1132884,), uint32)
  nbytes := 4.32 MB; cbytes := 4.40 MB; ratio: 0.98
  cparams := cparams(clevel=5, shuffle=1, cname='lz4', quantize=0)
  chunklen := 1048576; chunksize: 4194304; blocksize: 32768
  rootdir := 'D:\OpenSourceTradePlatform\atspy\rqalpha\bundle\indexes.bcolz\date'
  mode    := 'r'
[20130702 20130703 20130704 ..., 20170602 20170605 20170606]

In [34]:
table.cols[fields[0]][s:e]

array([12607820, 12416820, 12524930, ..., 30942253, 31021095, 30845401], dtype=uint32)

In [37]:
from collections import namedtuple
Rule = namedtuple('Rule', ['dtype', 'multiplier', 'round'])


class Converter(object):
    def __init__(self, rules):
        self._rules = rules

    def convert(self, name, data):
        try:
            r = self._rules[name]
        except KeyError:
            return data

        result = data * r.multiplier
        if r.round:
            result = np.round(result, r.round)

        return result

    def field_type(self, name, dt):
        try:
            return self._rules[name].dtype
        except KeyError:
            return dt

float64 = np.dtype('float64')

StockBarConverter = Converter({
    'open': Rule(float64, 1 / 10000.0, 2),
    'close': Rule(float64, 1 / 10000.0, 2),
    'high': Rule(float64, 1 / 10000.0, 2),
    'low': Rule(float64, 1 / 10000.0, 2),
    'limit_up': Rule(float64, 1/10000.0, 2),
    'limit_down': Rule(float64, 1/10000.0, 2),
    'volume': Rule(float64, 1, 0),
})


In [40]:
dtype = np.dtype([('datetime', np.uint64)] +
                         [(f, StockBarConverter.field_type(f, table.cols[f].dtype))
                          for f in fields])
dtype

dtype([('datetime', '<u8'), ('open', '<f8'), ('close', '<f8'), ('high', '<f8'), ('low', '<f8'), ('volume', '<f8'), ('total_turnover', '<u8')])

In [44]:
result = np.empty(shape=(e - s, ), dtype=dtype)
result

array([ (541215563516936384,  -5.10992911e+068,  -1.56207398e+091,   1.46594191e-190,   6.65072288e-145,   1.94620692e+169, 12436776856727494248),
       (3620989459858373760,   1.28866198e+193,  -2.44064717e+039,  -1.06509380e-290,  -5.93943461e+129,  -6.73119532e-170, 18199209151941243912),
       (3771253374147797122,  -2.18333928e+203,  -1.81022519e+212,   1.34277494e-144,  -3.55000709e+089,  -1.68440879e-096, 14738450706767044847),
       ...,
       (          128442568,   1.48539716e-313,   8.82780667e-272,   6.34590722e-316,   1.48539716e-313,   8.82794206e-272,            128442616),
       (30064773253,   8.82807745e-272,   6.34590959e-316,   1.48539716e-313,   8.82821284e-272,   7.52847305e-272, 30064773253),
       (                  0,   6.34591315e-316,   1.69759674e-313,   8.87577971e-272,   6.34598548e-316,   8.48798424e-314, 551696039594164226)], 
      dtype=[('datetime', '<u8'), ('open', '<f8'), ('close', '<f8'), ('high', '<f8'), ('low', '<f8'), ('volume', '<f8'), ('

In [55]:
for f in fields:
        result[f] = StockBarConverter.convert(f, table.cols[f][s:e])
        
result

array([ (541215563516936384,  1260.78,  1242.77,  1260.78,  1238.18,   8.16177000e+08, 4418452047),
       (3620989459858373760,  1241.68,  1251.94,  1258.58,  1235.75,   8.67865100e+08, 4916589314),
       (3771253374147797122,  1252.49,  1239.43,  1252.73,  1234.24,   7.92225400e+08, 4381370333),
       ...,
       (          128442568,  3094.23,  3105.54,  3110.39,  3081.85,   1.42504925e+10, 154454347995),
       (30064773253,  3102.11,  3091.66,  3105.51,  3084.83,   1.32570300e+10, 143429802002),
       (                  0,  3084.54,  3102.13,  3102.86,  3078.79,   1.13439963e+10, 127804624245)], 
      dtype=[('datetime', '<u8'), ('open', '<f8'), ('close', '<f8'), ('high', '<f8'), ('low', '<f8'), ('volume', '<f8'), ('total_turnover', '<u8')])

In [57]:
result['datetime'] = table.cols['date'][s:e]
result['datetime'] *= 1000000

result

array([ (20050104000000,  1260.78,  1242.77,  1260.78,  1238.18,   8.16177000e+08, 4418452047),
       (20050105000000,  1241.68,  1251.94,  1258.58,  1235.75,   8.67865100e+08, 4916589314),
       (20050106000000,  1252.49,  1239.43,  1252.73,  1234.24,   7.92225400e+08, 4381370333),
       ...,
       (20170602000000,  3094.23,  3105.54,  3110.39,  3081.85,   1.42504925e+10, 154454347995),
       (20170605000000,  3102.11,  3091.66,  3105.51,  3084.83,   1.32570300e+10, 143429802002),
       (20170606000000,  3084.54,  3102.13,  3102.86,  3078.79,   1.13439963e+10, 127804624245)], 
      dtype=[('datetime', '<u8'), ('open', '<f8'), ('close', '<f8'), ('high', '<f8'), ('low', '<f8'), ('volume', '<f8'), ('total_turnover', '<u8')])